# **UFLP**

In [1]:
nombre_archivo = "UFLP-1.txt"
with open(nombre_archivo, 'r') as f:
    lineas = f.readlines()

n, m = map(int, lineas[0].split())
# n son las ubicaciones potenciales
# m son los puntos de demanda

# inicializar listas para costos
f_j = [0] * (n + 1)  # costos fijos (indexados desde 1)

c_ij = [[0] * (n + 1) for _ in range(m + 1)]  # costos de asignación (indexados desde 1), por eso es m + 1

# leer los datos para cada ubicación potencial
for j in range(1, n + 1):
    datos = list(map(int, lineas[j].split()))
    idx = datos[0]  # indice de la ubicación
    f_j[idx] = datos[1]  # costo fijo
    # costos de asignación para cada punto de demanda
    for i in range(1, m + 1):
        c_ij[i][idx] = datos[i + 1]

print(f"Problema con {n} ubicaciones potenciales y {m} puntos de demanda")

Problema con 50 ubicaciones potenciales y 100 puntos de demanda


In [11]:
c_ij[1][2]

# que llegue a i desde j cuesta c_ij[1][2]

100

In [12]:
c_ij[1][3]

# que llegue a i desde j cuesta c_ij[1][3]

395

In [13]:
c_ij[4][1]

# que llegue a i desde j cuesta c_ij[4][1]

771

In [14]:
c_ij[100][1]

# que llegue a i desde j cuesta c_ij[100][1]

# que llegue a i desde j cuesta c_ij[i][j]

369

Algoritmo ADD

In [ ]:
print("Ejecutando algoritmo ADD:")
# ADD: Asignación Directa Determinística

# conjunto de ubicaciones seleccionadas
J_estrella = set()

# arreglo que indica la ubicación seleccionada donde se asigna la demanda del punto i
alloc_i = [0] * (m + 1)

# costo de la solución
f = 0
# se declara igual a cero para que no haya problemas con la suma de f_j[j] 

# calcular la función voraz adaptativa inicial
# la funcion voraz adaptativa es g(j) = Σ max(0, c_i,alloc_i - c_ij) - f_j
# significa
g = [0] * (n + 1)
for j in range(1, n + 1):
    g[j] = sum(c_ij[i][j] for i in range(1, m + 1)) + f_j[j]

# encontrar la mejor ubicación inicial (mínimo g(j))
j_estrella = 1 # decir que la mejor ubicación es la primera
for j in range(2, n + 1):  # recorrer las ubicaciones
    if g[j] < g[j_estrella]:    # si el costo de la ubicación actual es menor al costo de la mejor ubicación
        j_estrella = j  # la mejor ubicación es la actual

print(f"Ubicación inicial: {j_estrella}")
# actualizar conjuntos y costos
J_estrella.add(j_estrella)

# asignar demandas a cada ubicación seleccionada
for i in range(1, m + 1):
    alloc_i[i] = j_estrella

# el costo total es la suma de los costos fijos de las ubicaciones seleccionadas
f = g[j_estrella]   # g ya es la suma mas pequeña


# repetir hasta que no haya mejora
while True:
    # calcular la función voraz adaptativa para ubicaciones no seleccionadas
    g = [0] * (n + 1)
    for j in range(1, n + 1):
        if j not in J_estrella:
            # calcular g(j) = Σ max(0, c_i,alloc_i - c_ij) - f_j
            suma = 0
            for i in range(1, m + 1):
                diferencia = c_ij[i][alloc_i[i]] - c_ij[i][j]
                if diferencia > 0:
                    suma += diferencia
            g[j] = suma - f_j[j]
    
    # encontrar la mejor ubicación entre las no seleccionadas
    mejor_j = -1
    mejor_g = float('-inf')
    for j in range(1, n + 1):
        if j not in J_estrella and g[j] > mejor_g:
            mejor_j = j
            mejor_g = g[j]
    
    # si no hay mejora, terminar
    if mejor_j == -1 or mejor_g <= 0:
        break
    
    # actualizar conjuntos y costos
    j_estrella = mejor_j
    J_estrella.add(j_estrella)
    
    # reasignar demandas si es beneficioso
    for i in range(1, m + 1):
        if c_ij[i][j_estrella] < c_ij[i][alloc_i[i]]:
            alloc_i[i] = j_estrella
    
    # actualizar costo total
    f = f - mejor_g

# calcular costo inicial para verificar
costo_fijo_inicial = sum(f_j[j] for j in J_estrella)
costo_asignacion_inicial = sum(c_ij[i][alloc_i[i]] for i in range(1, m + 1))
costo_inicial = costo_fijo_inicial + costo_asignacion_inicial

print(f"Solución inicial:")
print(f"  Instalaciones seleccionadas: {sorted(J_estrella)}")
print(f"  Costo total: {costo_inicial}")



Ejecutando algoritmo ADD:
Ubicación inicial: 44
44
45854
Costo total inicial: 45854
Solución inicial:
  Instalaciones seleccionadas: [11, 14, 15, 33, 44]
  Costo total: 23468


Búsqueda Local

In [16]:
print("Aplicando búsqueda local:")

mejora = True
iteraciones = 0
max_iteraciones = 1000  
costo_actual = costo_inicial

while mejora and iteraciones < max_iteraciones:
    mejora = False
    iteraciones += 1
    
    # quito una instalación y la sustituyo por otra
    for j_quitar in list(J_estrella):
        # parar si solo hay una instalación
        if len(J_estrella) <= 1:
            break
            
        # intentar cada posible sustitución
        for j_añadir in range(1, n + 1):
            if j_añadir in J_estrella or j_añadir == j_quitar:
                continue  # ignorar si ya está seleccionada o es la misma
                
            # copias para evaluar la sustitución
            temp_J_estrella = J_estrella.copy()
            temp_J_estrella.remove(j_quitar)
            temp_J_estrella.add(j_añadir)
            temp_alloc_i = alloc_i.copy()
            
            # reasignar todos los puntos de demanda a la mejor instalación
            for i in range(1, m + 1):
                # Si el punto estaba asignado a la instalación que quitamos
                # o si la nueva instalación ofrece un costo menor, reasignar
                if temp_alloc_i[i] == j_quitar or c_ij[i][j_añadir] < c_ij[i][temp_alloc_i[i]]:
                    # buscar la mejor instalación entre todas las disponibles
                    mejor_instalacion = -1
                    mejor_costo = float('inf')
                    
                    for j in temp_J_estrella:
                        if c_ij[i][j] < mejor_costo:
                            mejor_costo = c_ij[i][j]
                            mejor_instalacion = j
                            
                    temp_alloc_i[i] = mejor_instalacion
            
            # nuevo costo total
            nuevo_costo_fijo = sum(f_j[j] for j in temp_J_estrella)
            nuevo_costo_asignacion = sum(c_ij[i][temp_alloc_i[i]] for i in range(1, m + 1))
            nuevo_costo = nuevo_costo_fijo + nuevo_costo_asignacion
            
            # si hay mejora, actualizar la solución
            if nuevo_costo < costo_actual:
                J_estrella = temp_J_estrella
                alloc_i = temp_alloc_i
                costo_actual = nuevo_costo
                mejora = True
                break
                
        if mejora:
            # si encontramos una mejora, pasamos a la siguiente iteración
            break

print(f"Solución mejorada:")
print(f"  Instalaciones seleccionadas: {sorted(J_estrella)}")
print(f"  Costo total: {costo_actual}")
 
if costo_actual < costo_inicial:
    mejora_porcentaje = (costo_inicial - costo_actual) / costo_inicial * 100
    print(f"Mejora lograda: {mejora_porcentaje:.2f}%")
else:
    print("No se logró mejorar la solución inicial")

Aplicando búsqueda local:
Solución mejorada:
  Instalaciones seleccionadas: [11, 14, 15, 33, 44]
  Costo total: 23468
No se logró mejorar la solución inicial
